В якості бази даних використовується MySQL із трьох master нод.

Налаштування знаходиться у [Docker Compose файлі](./compose.yaml). Після цього необхідно запустити [додатковий скрипт](./setup.sh). 

In [1]:
import mysql.connector

In [2]:
connection_to_node_1, connection_to_node_2, connection_to_node_3 = [
    mysql.connector.connect(
        database='test',
        user='myuser',
        password='mypass',
        host='localhost',
        port=3307 + i,
        autocommit=True,
    ) for i in range(3)
]

In [3]:
def write_message(message: str, connection):
    with connection.cursor() as cursor:
        cursor.execute('INSERT INTO messages (body) values (%s)', (message,))

In [4]:
def show_messages(connection):
    with connection.cursor() as cursor:
        cursor.execute('SELECT * FROM messages')
        print(cursor.fetchall())

In [5]:
with connection_to_node_1.cursor() as cursor:
    cursor.execute('DROP TABLE IF EXISTS messages')
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS messages (
            id int AUTO_INCREMENT,
            body varchar(255),
            PRIMARY KEY (id)
        )
    """)

Пишемо тестові дані в ноду 1 та перевіряємо, що вони доступні при читанні з ноди 2.

In [6]:
write_message('Message to node 1', connection_to_node_1)

In [7]:
show_messages(connection_to_node_2)

[(1, 'Message to node 1')]


Пишемо тестові дані в ноду 2 та перевіряємо, що вони доступні при читанні з ноди 3.

In [8]:
write_message('Message to node 2', connection_to_node_2)

In [9]:
show_messages(connection_to_node_3)

[(1, 'Message to node 1'), (2, 'Message to node 2')]


Перевіряємо, що вміст всіх трьох нод однаковий

In [10]:
show_messages(connection_to_node_1)
show_messages(connection_to_node_2)
show_messages(connection_to_node_3)

[(1, 'Message to node 1'), (2, 'Message to node 2')]
[(1, 'Message to node 1'), (2, 'Message to node 2')]
[(1, 'Message to node 1'), (2, 'Message to node 2')]
